In [72]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]") \
                    .getOrCreate()

df = spark.read.option("header",True) \
    .csv("DataSample.csv")
df.createOrReplaceTempView("data")

# clean data
sqlDF = spark.sql("SELECT * \
FROM data a \
WHERE a._ID IN \
  (SELECT _ID FROM \
     (SELECT \
      _ID, \
      ROW_NUMBER() OVER \
        (PARTITION BY data. TimeSt, data.Country, data.Province, data.City, data.Latitude, data.Longitude ORDER BY _ID) dup \
    FROM data) \
    WHERE dup < 2);")

sqlDF.write.option("header",True) \
 .csv("DataEdited")

In [107]:
import pyspark
from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from geopy import distance

spark = SparkSession.builder.master("local[*]") \
                    .getOrCreate()

df = spark.read.options(header='True',inferSchema='True',delimiter=',') \
    .csv("CleanedData.csv")
df.createOrReplaceTempView("data")

dfpoi = spark.read.options(header='True',inferSchema='True',delimiter=',') \
    .csv("POIList.csv")
dfpoi.createOrReplaceTempView("pois")


def dist(a,b,x,y):
    return distance.distance((a,b),(x,y)).km
udf_dist = udf(dist, FloatType())

# temp changed headers to make this work
distances = df.join(dfpoi).withColumn('distance', udf_dist(df.Latitude,df.Longitude,dfpoi.Latitude2,dfpoi.Longitude2))
min_distances = distances.groupBy('_ID').min('distance')
#final_distances = df.join(min_distances, df._ID == min_distances._ID).select(df["*"],min_distances["min(distance)"])
final_distances = distances.join(min_distances, distances._ID == min_distances._ID).select(distances["*"],min_distances["min(distance)"])
df_filtered = final_distances.filter(final_distances["distance"]==final_distances["min(distance)"])

df_filtered.write.option("header",True) \
 .csv("DataCalculated")

# pois.count()

# sqlDF = spark.sql()
# geopy.distance.distance(c1, c2)